In [4]:

import sys
import os
import math
import numpy as np
from numba import njit
from omr import *
from PIL import Image, ImageDraw, ImageFont
import time
TEMPLATE_DIR = "./templates/"
TEMPLATE_STAVE_DIST = 12
TREBLE_CLEF = ['E','D','C','B','A','G','F']
BASS_CLEF = ['G','F','E','D','C','B','A']

In [5]:
# timer added
def detect_symbols_using_hamming(I, T, score_thresh):
    '''Inputs are numpy array'''
    t_1 = time.time()
    hamming_score_array = compute_hamming(I, T)
    t_2 = time.time()    
    print("Time for computing hamming distance: ", t_2 - t_1)

    indices = np.where(hamming_score_array > hamming_score_array.max()-score_thresh)
    return indices, (hamming_score_array / T.size)

In [10]:
im = Image.open('./images/rach.png').convert(mode='L')
print("Detecting stave distance...")
t_0 = time.time()
staveDist, staves = detect_stave_distance(np.array(im, dtype=np.float64)/255)
imScaled, scale = scale_from_staves(im, staveDist)
print("Detecting notes...")
t_1 = time.time()

notes = detect_notes(np.array(imScaled)/255, scale, staves)
t_2 = time.time()
print("Time for computing hamming distance: ", t_2 - t_1)
print("Total computation time: ", t_2 - t_0)
visualized = visualize_notes(im, notes)
# visualized.show()
visualized.save("detected.png")
notes_to_txt(notes)
print("Done.")

Detecting stave distance...
Iteration 0/2480
Iteration 25/2480
Iteration 50/2480
Iteration 75/2480
Iteration 100/2480
Iteration 125/2480
Iteration 150/2480
Iteration 175/2480
Iteration 200/2480
Iteration 225/2480
Iteration 250/2480
Iteration 275/2480
Iteration 300/2480
Iteration 325/2480
Iteration 350/2480
Iteration 375/2480
Iteration 400/2480
Iteration 425/2480
Iteration 450/2480
Iteration 475/2480
Iteration 500/2480
Iteration 525/2480
Iteration 550/2480
Iteration 575/2480
Iteration 600/2480
Iteration 625/2480
Iteration 650/2480
Iteration 675/2480
Iteration 700/2480
Iteration 725/2480
Iteration 750/2480
Iteration 775/2480
Iteration 800/2480
Iteration 825/2480
Iteration 850/2480
Iteration 875/2480
Iteration 900/2480
Iteration 925/2480
Iteration 950/2480
Iteration 975/2480
Iteration 1000/2480
Iteration 1025/2480
Iteration 1050/2480
Iteration 1075/2480
Iteration 1100/2480
Iteration 1125/2480
Iteration 1150/2480
Iteration 1175/2480
Iteration 1200/2480
Iteration 1225/2480
Iteration 1250/24

In [ ]:
im

In [40]:
# temp = Image.open('./templates/template1.png').convert(mode='L')
# temp = Image.open('./images/music2.png').convert(mode='L')
# temp = np.array(temp) / 255
import cv2
# code adapted from https://medium.com/analytics-vidhya/2d-convolution-using-python-numpy-43442ff5f381
# processImage(Image) -> Image
# set the image as a grayscale image.
def sobel_gradient(image):
    sobkx1 = np.transpose(np.array([[1,2,1]]))
    sobkx2 = np.array([[-1,0,1]])
    sobky1 = np.transpose(np.array([[-1,0,1]]))
    sobky2 = np.array([[1,2,1]])
    a = convolve_separable(image, sobky1,sobky2)
    b = convolve_separable(image, sobkx1, sobkx2)

    c = np.sqrt(np.square(a) + np.square(b))
    c *= 255 / c.max()
    return c.astype(int)
def processImage(image):
    image = cv2.imread(image)
    image = cv2.cvtColor(src=image, code= cv2.COLOR_BGR2GRAY)
    return image
temp = processImage('./images/music2.png')
temp = sobel_gradient(temp)
# temp

In [3]:
Image.fromarray(temp).show()
# temp
# # print(np.array(temp))
# temp.sum()

NameError: name 'temp' is not defined

In [8]:
t = np.array([[10,  0,  0,  0],
       [10, 11,  1,  1],
       [10, 11, 12,  2]])
r = np.where(t < 10)
x,y = r
print(x,y)
for x,y in zip(r[0],r[1]):
    print(t[x][y])
t.shape

[0 0 0 1 1 2] [1 2 3 2 3 3]
0
0
0
1
1
2


(3, 4)

In [7]:
len(notes)
# notes[1]
notes
import pickle
def notes_to_txt(notes):
    with open("detected.txt", "wb") as outfile:
        # outfile.write("\n".join(str(item) for item in notes))
        # outfile.write(' '.join(map(str,notes)) + '\n')
        pickle.dump(notes, outfile, protocol = pickle.HIGHEST_PROTOCOL)
notes_to_txt(notes)
# def save_dtree(common_dict, filename):
#     with open(filename,'wb') as fp:
#         pickle.dump(common_dict, fp, protocol= pickle.HIGHEST_PROTOCOL)

In [48]:
# import the necessary packages
import numpy as np
# Malisiewicz et al.
def non_max_suppression_fast(boxes, overlapThresh):
	# if there are no boxes, return an empty list
	if len(boxes) == 0:
		return []
	# if the bounding boxes integers, convert them to floats --
	# this is important since we'll be doing a bunch of divisions
	if boxes.dtype.kind == "i":
		boxes = boxes.astype("float")
	# initialize the list of picked indexes	
	pick = []
	# grab the coordinates of the bounding boxes
	x1 = boxes[:,0]
	y1 = boxes[:,1]
	x2 = boxes[:,2]
	y2 = boxes[:,3]
	# compute the area of the bounding boxes and sort the bounding
	# boxes by the bottom-right y-coordinate of the bounding box
	area = (x2 - x1 + 1) * (y2 - y1 + 1)
	idxs = np.argsort(y2)
	# keep looping while some indexes still remain in the indexes
	# list
	while len(idxs) > 0:
		# grab the last index in the indexes list and add the
		# index value to the list of picked indexes
		last = len(idxs) - 1
		i = idxs[last]
		pick.append(i)
		# find the largest (x, y) coordinates for the start of
		# the bounding box and the smallest (x, y) coordinates
		# for the end of the bounding box
		xx1 = np.maximum(x1[i], x1[idxs[:last]])
		yy1 = np.maximum(y1[i], y1[idxs[:last]])
		xx2 = np.minimum(x2[i], x2[idxs[:last]])
		yy2 = np.minimum(y2[i], y2[idxs[:last]])
		# compute the width and height of the bounding box
		w = np.maximum(0, xx2 - xx1 + 1)
		h = np.maximum(0, yy2 - yy1 + 1)
		# compute the ratio of overlap
		overlap = (w * h) / area[idxs[:last]]
		# delete all indexes from the index list that have
		idxs = np.delete(idxs, np.concatenate(([last],
			np.where(overlap > overlapThresh)[0])))
	# return only the bounding boxes that were picked using the
	# integer data type
	return boxes[pick].astype("int")

In [3]:
notes

[[37.0, 319.0, 11.0, 17.0, 1, 'D', 0, 0.8874477319500725],
 [37.0, 320.0, 11.0, 17.0, 1, 'D', 0, 0.875107106069858],
 [37.0, 354.0, 11.0, 17.0, 1, 'D', 0, 0.8873196035255875],
 [37.0, 355.0, 11.0, 17.0, 1, 'D', 0, 0.874901343991513],
 [37.0, 389.0, 11.0, 17.0, 1, 'D', 0, 0.8871848959778937],
 [37.0, 390.0, 11.0, 17.0, 1, 'D', 0, 0.8747339053058573],
 [37.0, 755.0, 11.0, 17.0, 1, 'D', 0, 0.8841664764888862],
 [37.0, 756.0, 11.0, 17.0, 1, 'D', 0, 0.8704425899540902],
 [37.0, 790.0, 11.0, 17.0, 1, 'D', 0, 0.8840421310602461],
 [37.0, 791.0, 11.0, 17.0, 1, 'D', 0, 0.8702771250053971],
 [37.0, 825.0, 11.0, 17.0, 1, 'D', 0, 0.8839179501096861],
 [37.0, 826.0, 11.0, 17.0, 1, 'D', 0, 0.8701110021443832],
 [38.0, 318.0, 11.0, 17.0, 1, 'D', 0, 0.8840170481530132],
 [38.0, 319.0, 11.0, 17.0, 1, 'D', 0, 0.9301022436866118],
 [38.0, 320.0, 11.0, 17.0, 1, 'D', 0, 0.8604337698170379],
 [38.0, 353.0, 11.0, 17.0, 1, 'D', 0, 0.8840424600164057],
 [38.0, 354.0, 11.0, 17.0, 1, 'D', 0, 0.9299534732630604],